**Chapter 16 – Natural Language Processing with RNNs and Attention**

_This notebook contains all the sample code and solutions to the exercises in chapter 16._

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml3/blob/main/16_nlp_with_rnns_and_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml3/blob/main/16_nlp_with_rnns_and_attention.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Setup

This project requires Python 3.7 or above:

In [ ]:
import sys

assert sys.version_info >= (3, 7)

And TensorFlow ≥ 2.8:

In [ ]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

As we did in earlier chapters, let's define the default font sizes to make the figures prettier:

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

And let's create the `images/nlp` folder (if it doesn't already exist), and define the `save_fig()` function which is used through this notebook to save the figures in high-res for the book:

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "nlp"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

This chapter can be very slow without a GPU, so let's make sure there's one, or else issue a warning:

In [ ]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

# Sentiment Analysis

In [ ]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [ ]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8")[:200], "...")
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

**Warning**: the following cell will take a few minutes to run and the model will probably not learn anything because we didn't mask the padding tokens (that's the point of the next section).

In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 52s 63ms/step - loss: 0.6934 - accuracy: 0.5010 - val_loss: 0.6931 - val_accuracy: 0.5020
Epoch 2/5
704/704 [==============================] - 25s 36ms/step - loss: 0.6926 - accuracy: 0.5038 - val_loss: 0.6939 - val_accuracy: 0.4992
Epoch 3/5
704/704 [==============================] - 24s 34ms/step - loss: 0.6916 - accuracy: 0.5019 - val_loss: 0.6946 - val_accuracy: 0.5000
Epoch 4/5
704/704 [==============================] - 24s 34ms/step - loss: 0.6946 - accuracy: 0.5025 - val_loss: 0.6941 - val_accuracy: 0.4976
Epoch 5/5
704/704 [==============================] - 23s 33ms/step - loss: 0.6907 - accuracy: 0.5080 - val_loss: 0.6953 - val_accuracy: 0.5008


## Masking

**Warning**: the following cell will take a while to run (possibly 30 minutes if you are not using a GPU).

In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 50s 64ms/step - loss: 0.5102 - accuracy: 0.7419 - val_loss: 0.3519 - val_accuracy: 0.8584
Epoch 2/5
704/704 [==============================] - 28s 39ms/step - loss: 0.3288 - accuracy: 0.8620 - val_loss: 0.3199 - val_accuracy: 0.8688
Epoch 3/5
704/704 [==============================] - 25s 35ms/step - loss: 0.2892 - accuracy: 0.8783 - val_loss: 0.3262 - val_accuracy: 0.8624
Epoch 4/5
704/704 [==============================] - 25s 36ms/step - loss: 0.2660 - accuracy: 0.8905 - val_loss: 0.3048 - val_accuracy: 0.8640
Epoch 5/5
704/704 [==============================] - 26s 37ms/step - loss: 0.2496 - accuracy: 0.8997 - val_loss: 0.3254 - val_accuracy: 0.8592


Der folgende Code soll nur dabei helfen die gelernten Embeddinigs abzuspeichern, um diese visualisieren zu können.

In [ ]:
# Angenommen, `embedding_layer` ist Ihre Embedding-Schicht im Modell
embeddings = model.layers[1].get_weights()[0]

# Wörter/Begriffe zu den Embeddings holen
words = text_vec_layer.get_vocabulary()

# Embeddings und Wörter speichern
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(words):
  vec = embeddings[num]  # das num-te Wort im Vokabular holen
  out_m.write(word + "\n")  # Wort schreiben
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")  # Embedding von dem Wort schreiben
out_v.close()
out_m.close()

**Extra material: using ragged tensors**

In [ ]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size, ragged=True)
text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])

<tf.RaggedTensor [[86, 18], [11, 7, 1, 116, 217]]>

In [ ]:
text_vec_layer(["Great movie!", "This is DiCaprio's best role."])

<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[ 86,  18,   0,   0,   0],
       [ 11,   7,   1, 116, 217]])>

**Warning**: the following cell will take a while to run (possibly 30 minutes if you are not using a GPU).

In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 50s 68ms/step - loss: 0.5679 - accuracy: 0.7077 - val_loss: 0.5084 - val_accuracy: 0.7520
Epoch 2/5
704/704 [==============================] - 28s 40ms/step - loss: 0.3692 - accuracy: 0.8380 - val_loss: 0.3311 - val_accuracy: 0.8568
Epoch 3/5
704/704 [==============================] - 28s 39ms/step - loss: 0.3079 - accuracy: 0.8700 - val_loss: 0.3165 - val_accuracy: 0.8600
Epoch 4/5
704/704 [==============================] - 27s 38ms/step - loss: 0.2866 - accuracy: 0.8800 - val_loss: 0.3091 - val_accuracy: 0.8644
Epoch 5/5
704/704 [==============================] - 26s 37ms/step - loss: 0.2694 - accuracy: 0.8889 - val_loss: 0.3173 - val_accuracy: 0.8664


## Reusing Pretrained Embeddings and Language Models

**Warning**: the following cell will take a while to run (possibly an hour if you are not using a GPU).

In [ ]:
import os
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2",
                   trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 135s 188ms/step - loss: 0.3841 - accuracy: 0.8285 - val_loss: 0.2836 - val_accuracy: 0.8792
Epoch 2/5
704/704 [==============================] - 79s 112ms/step - loss: 0.1605 - accuracy: 0.9405 - val_loss: 0.3070 - val_accuracy: 0.8748
Epoch 3/5
704/704 [==============================] - 51s 72ms/step - loss: 0.0655 - accuracy: 0.9792 - val_loss: 0.4005 - val_accuracy: 0.8724
Epoch 4/5
704/704 [==============================] - 34s 48ms/step - loss: 0.0209 - accuracy: 0.9955 - val_loss: 0.5085 - val_accuracy: 0.8720
Epoch 5/5
704/704 [==============================] - 31s 44ms/step - loss: 0.0047 - accuracy: 0.9997 - val_loss: 0.6077 - val_accuracy: 0.8688
